# import thư viện

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

numpy xử lý số, mảng

tensorflow dùng để tạo mô hình AI

Model để tạo mô hình kiểu encoder–decoder

Input khai báo đầu vào

LSTM lớp mạng xử lý chuỗi (quan trọng)

Dense lớp dự đoán từ cuối

# Tạo dataset 20 câu

In [22]:
vi_sentences = [
    "tôi ăn cơm",
    "tôi uống nước",
    "tôi đi học",
    "tôi ngủ",
    "tôi chơi game",
    "bạn ăn cơm",
    "bạn uống nước",
    "bạn đi học",
    "bạn ngủ",
    "bạn chơi game",
    "anh ấy ăn cơm",
    "anh ấy uống nước",
    "anh ấy đi học",
    "anh ấy ngủ",
    "anh ấy chơi game",
    "cô ấy ăn cơm",
    "cô ấy uống nước",
    "cô ấy đi học",
    "cô ấy ngủ",
    "cô ấy chơi game"
]

en_sentences = [
    "i eat rice",
    "i drink water",
    "i go to school",
    "i sleep",
    "i play games",
    "you eat rice",
    "you drink water",
    "you go to school",
    "you sleep",
    "you play games",
    "he eats rice",
    "he drinks water",
    "he goes to school",
    "he sleeps",
    "he plays games",
    "she eats rice",
    "she drinks water",
    "she goes to school",
    "she sleeps",
    "she plays games"
]

# PHẦN 3: Thêm token START và END

In [23]:
en_input = ["<start> " + s for s in en_sentences]
en_output = [s + " <end>" for s in en_sentences]

Trong decoder:

phải biết khi nào bắt đầu sinh câu

phải biết khi nào kết thúc câu

# PHẦN 4: Tokenizer (chuyển chữ thành số)

In [24]:
tokenizer_vi = tf.keras.preprocessing.text.Tokenizer()
tokenizer_en = tf.keras.preprocessing.text.Tokenizer()

tokenizer_vi.fit_on_texts(vi_sentences)
tokenizer_en.fit_on_texts(en_input + en_output)

vi_seq = tokenizer_vi.texts_to_sequences(vi_sentences)
en_in_seq = tokenizer_en.texts_to_sequences(en_input)
en_out_seq = tokenizer_en.texts_to_sequences(en_output)

max_vi = max(len(s) for s in vi_seq)
max_en = max(len(s) for s in en_in_seq)

vi_seq = tf.keras.preprocessing.sequence.pad_sequences(vi_seq, maxlen=max_vi)
en_in_seq = tf.keras.preprocessing.sequence.pad_sequences(en_in_seq, maxlen=max_en)
en_out_seq = tf.keras.preprocessing.sequence.pad_sequences(en_out_seq, maxlen=max_en)


Giải thích:

Máy không hiểu chữ → phải chuyển thành số

Ví dụ:

"tôi ăn cơm" → [1, 2, 3]
"i eat rice" → [1, 4, 5]


Tokenizer tạo từ điển:

"tôi" → 1
"ăn" → 2
"cơm" → 3

In [25]:
latent_dim = 64

In [26]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = tf.keras.layers.Embedding(len(tokenizer_vi.word_index)+1, 64)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

In [27]:
# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb = tf.keras.layers.Embedding(len(tokenizer_en.word_index)+1, 64)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])
decoder_dense = Dense(len(tokenizer_en.word_index)+1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

print("Training...")
model.fit([vi_seq, en_in_seq], np.expand_dims(en_out_seq, -1), epochs=200)

Training...
Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 3.0907
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 3.0824
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 3.0742
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 3.0658
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 3.0572
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 3.0480
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 3.0383
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 3.0277
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 3.0163
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 3.0038
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 2.9899
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 2.9745
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 2.9573
Epoch 14/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 2.9380
Epoch 15/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 2.9162
Epoch 16/

In [28]:
# ======== BUILD INFERENCE MODEL ========

encoder_model = Model(encoder_inputs, [state_h, state_c])

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
dec_emb2 = dec_emb
decoder_outputs2, h2, c2 = decoder_lstm(dec_emb2,
                                        initial_state=[decoder_state_input_h,
                                                       decoder_state_input_c])
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2, h2, c2]
)


In [30]:
# ======== FUNCTION DỊCH ========

word_index_en = tokenizer_en.word_index
index_word_en = {v:k for k,v in word_index_en.items()}

def translate(sentence):
    seq = tokenizer_vi.texts_to_sequences([sentence])
    seq = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_vi)

    h, c = encoder_model.predict(seq)

    target_seq = np.array([[word_index_en["start"]]])

    result = ""

    for _ in range(6):
        output, h, c = decoder_model.predict([target_seq, h, c])
        idx = np.argmax(output[0, -1, :])
        word = index_word_en.get(idx, "")

        if word == "end":
            break

        result += word + " "

        target_seq = np.array([[idx]])

    return result.strip()

In [42]:
print("\nDịch thử:")
print("Output:", translate("cô chơi garme"))


Dịch thử:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Output: i go to school
